In [94]:
import pandas as pd

import re
from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords 

import requests
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from sklearn.cross_validation import cross_val_score

pd.options.display.max_colwidth = 2000
pd.options.display.max_rows = 999

## Prior to this modeling I randomly selected 150 columns and assign labels to each row.

# 1- Reading Sample Dataset with labels assigned

In [166]:
dataset = pd.read_excel('SAMPLEDATASET1.xlsx')

In [167]:
dataset.head()

,name,username,description,location,followers,numberstatuses,Date & Time,Date,Time,tweets,labels
0,War BreakingNews,warrnews,we provide fresh news from every battlefield,world,6948,6265,2/8/2016 14:43,2016-02-08,14:43:00,"RT @Abduhark: Badr militia formed by Iran in1982, sided with America in2003 to massacre Iraqi Sunnis, sided with Russia in2015 to massacre …",O
1,☝️,melvynlion,"If you have trust in Allah, you have no need to worry, as one way or another the sun will shine again.!!!",NaN,81,478,5/12/2016 18:15,2016-05-12,18:15:00,@spicylatte123 @kafirkaty @ScotsmanInfidel @sassysassyred same here,O
2,☝️,melvynlion,"If you have trust in Allah, you have no need to worry, as one way or another the sun will shine again.!!!",NaN,43,137,4/20/2016 0:57,2016-04-20,00:57:00,RT @sparksofirhabi4: Complaining of doubts from Shaytaan https://t.co/rTPS0IAL5k,I
3,al-Nusra,al_nusra,NaN,NaN,350,190,10/9/2015 14:27,2015-10-09,14:27:00,هـام | #سوريا | مراسل العربي الجديد في طهران يأكد مقتل الجنرال ( حسين همداني ) في كمين للدولة الإسلامية في #حلب\n\nhttp://t.co/GfGsVKxp80,N
4,أبو أسامة الصومالي,CXaafada2,Degree from Tableeq(UK) | Masters from Alshabaab(Somalia) | PhD Research Fellow at Islamic State(Yemen). #IslamicState #IslamicPolitics #Somalia #Yemen,Wilayat Hadramaut,866,370,1/22/2016 17:04,2016-01-22,17:04:00,GLAD TIDINGS: can't wait to see Kenyan POW captured by shabaab to be paraded soon.,N


# Features Engineering

### scikit-learn works with numerical data
### Adding columns for RT: Retweets, URL: containing URL links, and Userstart: tweets starting by a @username 
### to replace the "tweets" column in our first approach to model the data.

In [95]:
sample = pd.read_excel('SAMPLEDATASET3RTURLUSER.xlsx')

In [96]:
sample.head()

,name,username,description,location,followers,numberstatuses,Date & Time,Date,Time,tweets,RT,URL,Userstart,labels
0,War BreakingNews,warrnews,we provide fresh news from every battlefield,world,6948,6265,2/8/2016 14:43,2016-02-08,14:43:00,"RT @Abduhark: Badr militia formed by Iran in1982, sided with America in2003 to massacre Iraqi Sunnis, sided with Russia in2015 to massacre …",1,0,0,O
1,☝️,melvynlion,"If you have trust in Allah, you have no need to worry, as one way or another the sun will shine again.!!!",NaN,81,478,5/12/2016 18:15,2016-05-12,18:15:00,@spicylatte123 @kafirkaty @ScotsmanInfidel @sassysassyred same here,0,0,1,O
2,☝️,melvynlion,"If you have trust in Allah, you have no need to worry, as one way or another the sun will shine again.!!!",NaN,43,137,4/20/2016 0:57,2016-04-20,00:57:00,RT @sparksofirhabi4: Complaining of doubts from Shaytaan https://t.co/rTPS0IAL5k,1,1,0,I
3,al-Nusra,al_nusra,NaN,NaN,350,190,10/9/2015 14:27,2015-10-09,14:27:00,هـام | #سوريا | مراسل العربي الجديد في طهران يأكد مقتل الجنرال ( حسين همداني ) في كمين للدولة الإسلامية في #حلب\n\nhttp://t.co/GfGsVKxp80,0,1,0,N
4,أبو أسامة الصومالي,CXaafada2,Degree from Tableeq(UK) | Masters from Alshabaab(Somalia) | PhD Research Fellow at Islamic State(Yemen). #IslamicState #IslamicPolitics #Somalia #Yemen,Wilayat Hadramaut,866,370,1/22/2016 17:04,2016-01-22,17:04:00,GLAD TIDINGS: can't wait to see Kenyan POW captured by shabaab to be paraded soon.,0,0,0,N


In [97]:
sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 14 columns):
name              150 non-null object
username          150 non-null object
description       125 non-null object
location          108 non-null object
followers         150 non-null int64
numberstatuses    150 non-null int64
Date & Time       150 non-null object
Date              150 non-null datetime64[ns]
Time              150 non-null object
tweets            150 non-null object
RT                150 non-null int64
URL               150 non-null int64
Userstart         150 non-null int64
labels            150 non-null object
dtypes: datetime64[ns](1), int64(5), object(8)
memory usage: 16.5+ KB


# A.

# First Approach: Classification Using Sklearn

In [98]:
columns = ['name', 'username', 'description', 'location', 'followers', 'numberstatuses', 'Date', 'Time', 'tweets', 'RT', 'URL', 'Userstart', 'labels']
sample = sample[columns]

In [99]:
sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 13 columns):
name              150 non-null object
username          150 non-null object
description       125 non-null object
location          108 non-null object
followers         150 non-null int64
numberstatuses    150 non-null int64
Date              150 non-null datetime64[ns]
Time              150 non-null object
tweets            150 non-null object
RT                150 non-null int64
URL               150 non-null int64
Userstart         150 non-null int64
labels            150 non-null object
dtypes: datetime64[ns](1), int64(5), object(7)
memory usage: 15.3+ KB


In [100]:
sample.head()

,name,username,description,location,followers,numberstatuses,Date,Time,tweets,RT,URL,Userstart,labels
0,War BreakingNews,warrnews,we provide fresh news from every battlefield,world,6948,6265,2016-02-08,14:43:00,"RT @Abduhark: Badr militia formed by Iran in1982, sided with America in2003 to massacre Iraqi Sunnis, sided with Russia in2015 to massacre …",1,0,0,O
1,☝️,melvynlion,"If you have trust in Allah, you have no need to worry, as one way or another the sun will shine again.!!!",NaN,81,478,2016-05-12,18:15:00,@spicylatte123 @kafirkaty @ScotsmanInfidel @sassysassyred same here,0,0,1,O
2,☝️,melvynlion,"If you have trust in Allah, you have no need to worry, as one way or another the sun will shine again.!!!",NaN,43,137,2016-04-20,00:57:00,RT @sparksofirhabi4: Complaining of doubts from Shaytaan https://t.co/rTPS0IAL5k,1,1,0,I
3,al-Nusra,al_nusra,NaN,NaN,350,190,2015-10-09,14:27:00,هـام | #سوريا | مراسل العربي الجديد في طهران يأكد مقتل الجنرال ( حسين همداني ) في كمين للدولة الإسلامية في #حلب\n\nhttp://t.co/GfGsVKxp80,0,1,0,N
4,أبو أسامة الصومالي,CXaafada2,Degree from Tableeq(UK) | Masters from Alshabaab(Somalia) | PhD Research Fellow at Islamic State(Yemen). #IslamicState #IslamicPolitics #Somalia #Yemen,Wilayat Hadramaut,866,370,2016-01-22,17:04:00,GLAD TIDINGS: can't wait to see Kenyan POW captured by shabaab to be paraded soon.,0,0,0,N


# Selecting columns

### I dropped description and location because they were mostly fake and with no relevant information.  ISIS fans hide their locations for security reasons
  

In [152]:
columns2 = ['followers', 'numberstatuses', 'RT', 'URL', 'Userstart']

In [153]:
X2 = sample[columns2]

In [154]:
X2

,followers,numberstatuses,RT,URL,Userstart
0,6948,6265,1,0,0
1,81,478,0,0,1
2,43,137,1,1,0
3,350,190,0,1,0
4,866,370,0,0,0
5,30487,17137,0,1,0
6,1667,13960,0,0,1
7,131,26,1,0,0
8,83,29,0,1,0
9,74,121,0,1,0


In [155]:
X2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
followers         150 non-null int64
numberstatuses    150 non-null int64
RT                150 non-null int64
URL               150 non-null int64
Userstart         150 non-null int64
dtypes: int64(5)
memory usage: 5.9 KB


In [156]:
X2.describe()

,followers,numberstatuses,RT,URL,Userstart
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,4677.993333,5456.833333,0.293333,0.466667,0.186667
std,9539.884731,7662.036505,0.456815,0.500559,0.390949
min,21.000000,26.000000,0.000000,0.000000,0.000000
25%,264.750000,189.250000,0.000000,0.000000,0.000000
50%,900.000000,926.500000,0.000000,0.000000,0.000000
75%,2119.000000,7469.250000,1.000000,1.000000,0.000000
max,34692.000000,33091.000000,1.000000,1.000000,1.000000


# Getting Labels into a Dictionary

In [157]:
categories = sample["labels"].unique()
category_dict = {value:index for index, value in enumerate(categories)}
results = sample["labels"].map(category_dict)
category_dict


{u'I': 1, u'N': 2, u'O': 0}

# Splitting the dataset into training and testing sets

In [158]:
x2_train,x2_test, y2_train,y2_test = train_test_split(X2,results , test_size=0.3, random_state=1)


# Using Models to predict labels

## a- Naive Bayes Classifier

In [160]:
clf_1 = MultinomialNB()
clf_1.fit(x2_train, y2_train)
clf_1.score(x2_test, y2_test)

0.42222222222222222

## b- K Neighbors Classifier

In [161]:
clf_2 = KNeighborsClassifier()
clf_2.fit(x2_train, y2_train)
clf_2.score(x2_test, y2_test)

0.48888888888888887

## c- Random Forest Classifier

In [164]:
clf_3 = RandomForestClassifier()
clf_3.fit(x2_train, y2_train)
clf_3.score(x2_test, y2_test)

0.62222222222222223

## d- Gradient Boosting

In [165]:
clf_4 = GradientBoostingClassifier()
clf_4.fit(x2_train, y2_train)
clf_4.score(x2_test, y2_test)

0.59999999999999998

# Cross Validation

## a- Naive Bayes Classifier

In [169]:
cross_val_score(clf_1, X2, results , cv=10).mean()

0.41076923076923072

## b- K Neighbors Classifier

In [170]:
cross_val_score(clf_2, X2, results , cv=10).mean()

0.46554487179487181

## c- Random Forest Classifier

In [171]:
cross_val_score(clf_3, X2, results , cv=10).mean()

0.5625641025641025

## d- Gradient Boosting

In [172]:
cross_val_score(clf_4, X2, results , cv=10).mean()

0.55342948717948715

# B.

# Second Approach: Using Tweets to predict the labels

# Count Vectorizer to get feature names

In [101]:
corpus = sample["tweets"]
vectorizer = CountVectorizer(min_df=1)
X = vectorizer.fit_transform(corpus).toarray()
print X.shape
X


(150L, 1231L)


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [102]:
vectorizer.get_feature_names()[:25]

[u'000',
 u'06kill56',
 u'10',
 u'100',
 u'1000',
 u'100s',
 u'12',
 u'140',
 u'14th',
 u'16',
 u'18',
 u'19',
 u'1gk4bgcrih',
 u'1yc9z7pkke',
 u'2003',
 u'2016',
 u'22',
 u'24',
 u'25',
 u'26',
 u'2d6rlb3g9a',
 u'2dvc8jrzyj',
 u'2mss51rhni',
 u'2skgawrtn7',
 u'34']

# Getting Labels into a Dictionary

In [103]:
categories = sample["labels"].unique()
category_dict = {value:index for index, value in enumerate(categories)}
results = sample["labels"].map(category_dict)
category_dict


{u'I': 1, u'N': 2, u'O': 0}

In [104]:
print "corpus size: %s" % len(vectorizer.get_feature_names())

corpus size: 1231


In [105]:
print results

0      0
1      0
2      1
3      2
4      2
5      2
6      0
7      2
8      1
9      2
10     0
11     2
12     0
13     2
14     2
15     0
16     0
17     0
18     0
19     0
20     1
21     2
22     0
23     0
24     0
25     0
26     2
27     0
28     0
29     0
30     0
31     2
32     0
33     0
34     2
35     2
36     0
37     0
38     0
39     0
40     2
41     2
42     1
43     2
44     0
45     2
46     2
47     2
48     1
49     0
50     0
51     2
52     2
53     2
54     0
55     2
56     2
57     0
58     2
59     2
60     2
61     2
62     2
63     0
64     2
65     0
66     0
67     0
68     2
69     2
70     0
71     2
72     2
73     0
74     2
75     0
76     2
77     2
78     0
79     2
80     2
81     2
82     0
83     0
84     0
85     0
86     0
87     0
88     0
89     2
90     2
91     2
92     2
93     1
94     2
95     2
96     0
97     1
98     2
99     0
100    0
101    0
102    0
103    2
104    1
105    2
106    2
107    2
108    2
109    2
110    0
1

In [106]:
len(results)

150

# Splitting the dataset into training and testing sets

In [107]:
x_train,x_test, y_train,y_test = train_test_split(X, results, test_size=0.3, random_state=1)


# Using Models to predict labels

## a- Naive Bayes Classifier

In [108]:
clf1 = MultinomialNB()
clf1.fit(x_train, y_train)
clf1.score(x_test, y_test)

0.68888888888888888

## b- K Neighbors Classifier

In [124]:
clf2 = KNeighborsClassifier()
clf2.fit(x_train, y_train)
clf2.score(x_test, y_test)

0.42222222222222222

## c- Random Forest Classifier

In [127]:
clf3 = RandomForestClassifier()
clf3.fit(x_train, y_train)
clf3.score(x_test, y_test)

0.73333333333333328

## d- Gradient Boosting

In [128]:
clf4 = GradientBoostingClassifier()
clf4.fit(x_train, y_train)
clf4.score(x_test, y_test)

0.80000000000000004

# Function to Print Classisers Predictions

In [129]:
def run_classifiers(text):
    vec_text = vectorizer.transform(text).toarray()
    NaiveBayes = category_dict.keys()[category_dict.values().index(clf1.predict(vec_text)[0])]
    KNeighbors = category_dict.keys()[category_dict.values().index(clf2.predict(vec_text)[0])]
    RandForest = category_dict.keys()[category_dict.values().index(clf3.predict(vec_text)[0])]
    GradBoosti = category_dict.keys()[category_dict.values().index(clf4.predict(vec_text)[0])]
    print "Predictions: Naive Bayes: " + NaiveBayes + " KNeighbors: " + KNeighbors + " Random Forest: "+ RandForest + " Gradient Boosting: " + GradBoosti

# Testing Classifiers with tweets outside the sample dataset

In [130]:
text = ["Who will win the Superbowl?"]

In [131]:
run_classifiers(text)

Predictions: Naive Bayes: O KNeighbors: O Random Forest: O Gradient Boosting: O


In [132]:
text = ["capture more than 60 soldiers of SAA including 4 Russians,destruction of 2 tanks near d village alSalhiyeh in d c'side of alsafirah "]

In [133]:
run_classifiers(text)

Predictions: Naive Bayes: N KNeighbors: O Random Forest: N Gradient Boosting: N


In [134]:
text = ["RT @aasiyehandrabi: ##jihad becomes obligatory on Muslim ummah when the honour of  Muslim women is endangered.where is Ibni qasim Aafiya is…"]

In [135]:
run_classifiers(text)

Predictions: Naive Bayes: O KNeighbors: O Random Forest: N Gradient Boosting: I


In [136]:
text = ["The hypocrites say the Mujahideen are breaking up families while Allah says... https://t.co/wY1anC9BVF"]

In [137]:
run_classifiers(text)

Predictions: Naive Bayes: N KNeighbors: O Random Forest: N Gradient Boosting: N


In [138]:
text = ["هـام | #سوريا | مراسل العربي الجديد في طهران يأكد مقتل الجنرال ( حسين همداني ) في كمين للدولة الإسلامية في #حلب\n\nhttp://t.co/GfGsVKxp80"]


In [139]:
run_classifiers(text)

Predictions: Naive Bayes: N KNeighbors: O Random Forest: N Gradient Boosting: N


# Cross Validation

## a- Naive Bayes Classifier

In [140]:
cross_val_score(clf1, X, results , cv=10).mean()

0.69464743589743594

## b- K Neighbors Classifier

In [141]:
cross_val_score(clf2, X, results , cv=10).mean()

0.51301282051282049

## c- Random Forest Classifier

In [142]:
cross_val_score(clf3, X, results , cv=10).mean()

0.66150641025641022

## d- Gradient Boosting

In [143]:
cross_val_score(clf4, X, results , cv=10).mean()

0.73964743589743587

# Accuracy Score

## a- Naive Bayes Classifier

In [144]:
accuracy_score(y_test, clf1.predict(x_test))

0.68888888888888888

## b- K Neighbors Classifier

In [145]:
accuracy_score(y_test, clf2.predict(x_test))

0.42222222222222222

## c- Random Forest Classifier

In [146]:
accuracy_score(y_test, clf3.predict(x_test))

0.73333333333333328

## d- Gradient Boosting

In [147]:
accuracy_score(y_test, clf4.predict(x_test))

0.80000000000000004